In [462]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [463]:
pt_low, pt_high = 1, 16
phi_v_low, phi_v_high = 2, 31

In [464]:
isigma = 2
N_centr = 6
N_options = 6
centralities = ["0-88%","0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [0,4,0,0,1,1,2,2,3,3,4,4]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAU_first_test_19060_530runs.root"#"m_ee_Run14AuAU_first_test_19060_100runs.root"

In [465]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(1,N_options+1):
    histo_name = f"inv_m_ee_V{iOpt}_"
    histo_name_like = f"phi_ee_V{iOpt}_"
    print(histo_name)
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = infile.Get("c00_z00_r00/"+histo_name+"FG12")
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_name+"BG12")
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = infile.Get("c00_z00_r00/"+histo_name_like+"FG11")
        hist_FG_like.Add(hist_FG_like,-1)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = infile.Get("c00_z00_r00/"+histo_name_like+"BG11")
        hist_BG_like.Add(hist_BG_like,-1)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(2):
                hist_FG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name+"FG12"))
                hist_BG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name+"BG12"))
                hist_FG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"FG11"))
                hist_BG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"BG11"))
                hist_FG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"FG22"))
                hist_BG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"BG22"))
                hist_stat.Add(infile.Get(f"c0{i}_z0{j}_r00/PoolStatistics"))
        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

infile.Close()

inv_m_ee_V1_
inv_m_ee_V2_
inv_m_ee_V3_
inv_m_ee_V4_
inv_m_ee_V5_
inv_m_ee_V6_


In [466]:
def emc_fitter(InHistFG, InHistBG, centr, seed=1, sigma=2, fits_mean=0, fits_sigma=0):

    eta_FG_porj = InHistFG[centr].ProjectionY(f"FG{seed*10+centr}",pt_low,pt_high,phi_v_low,phi_v_high)
    eta_BG_porj = InHistBG[centr].ProjectionY(f"BG{seed*10+centr}",pt_low,pt_high,phi_v_low,phi_v_high)

    left = eta_FG_porj.FindBin(1.3)
    right = eta_FG_porj.FindBin(1.8)
    
    scale = eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1)
    
    return scale


In [467]:
def emc_centr_fitter(InHistFG, InHistBG, centralities, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    hist_ser = []
    for centr in centralities:
        hist_ser.append(emc_fitter(InHistFG, InHistBG, centr, seed, sigma, fits_mean, fits_sigma))
    return hist_ser

In [468]:
import multiprocess as mp

Ntr = N_options

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_ee_FG_read[i], hists_ee_BG_read[i], [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [469]:
iOption = 5
rebin = 5
new_pt_array = array( 'd', [0.,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])
N_new_bins = len(new_pt_array)-1
print(N_new_bins)

33


In [470]:

c3 = root.TCanvas(f"c2",f"c2",500*3,500*2)
c3.Divide(3,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ee_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ee_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icentr])
    proj.Rebin(1)
    proj_BG.Rebin(1)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.SetMaximum(proj.GetMaximum()*10)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

In [471]:
c3 = root.TCanvas(f"c2",f"c2",500*3,500*2)
c3.Divide(3,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icentr])
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

    print(round(hist_stat_read[iOption][icentr].GetBinContent(2)/1e6,3),"*10^6")

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

6281.298 *10^6
1316.963 *10^6
1302.133 *10^6
1343.463 *10^6
1403.787 *10^6
914.952 *10^6


In [472]:
c3 = root.TCanvas(f"c2",f"c2",500*3,500*2)
c3.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    projs.append(root.TH1D(f"my{icentr}_{iOption}",f"my{icentr}_{iOption}",N_new_bins,new_pt_array))
    proj = projs[-1]
    projs_BG.append(root.TH1D(f"my{icentr}_{iOption}_BG",f"my{icentr}_{iOption}_BG",N_new_bins,new_pt_array))
    proj_BG = projs_BG[-1]
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)

    for i in range(1,1+proj.GetNbinsX()):
        for j in range(proj1.FindBin(proj.GetBinLowEdge(i)),1+proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i))):
            proj.SetBinContent(i,proj.GetBinContent(i)+proj1.GetBinContent(j))
            proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)+proj1_BG.GetBinContent(j))
        proj.SetBinContent(i,proj.GetBinContent(i)/proj.GetBinWidth(i))
        proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)/proj_BG.GetBinWidth(i))
        
    proj_BG.Scale(output_array[iOption][icentr])
    #proj.Rebin(rebin)
    #proj_BG.Rebin(rebin)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

In [473]:
c3 = root.TCanvas(f"c2",f"c2",500*3,500*2)
c3.Divide(3,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icentr])
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    proj.Add(proj_BG,-0.9)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

    print(round(hist_stat_read[iOption][icentr].GetBinContent(2)/1e6,3),"*10^6")

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

6281.298 *10^6
1316.963 *10^6
1302.133 *10^6
1343.463 *10^6
1403.787 *10^6
914.952 *10^6


In [474]:
c3 = root.TCanvas(f"c2",f"c2",500*3,500*2)
Opt_array=[4,5,4,3,2,0]
sub_koeff=[0.92,0.8,0.85,0.8,0.75,0.5]
c3.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    iOption = Opt_array[icentr]
    c3.cd(icentr+1)
    projs.append(root.TH1D(f"my{icentr}_{iOption}",f"my{icentr}_{iOption}",N_new_bins,new_pt_array))
    proj = projs[-1]
    projs_BG.append(root.TH1D(f"my{icentr}_{iOption}_BG",f"my{icentr}_{iOption}_BG",N_new_bins,new_pt_array))
    proj_BG = projs_BG[-1]
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)

    for i in range(1,1+proj.GetNbinsX()):
        for j in range(proj1.FindBin(proj.GetBinLowEdge(i)),1+proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i))):
            proj.SetBinContent(i,proj.GetBinContent(i)+proj1.GetBinContent(j))
            proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)+proj1_BG.GetBinContent(j))
        proj.SetBinContent(i,proj.GetBinContent(i)/proj.GetBinWidth(i))
        proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)/proj_BG.GetBinWidth(i))
        
    proj_BG.Scale(output_array[iOption][icentr])
    proj.Add(proj_BG,-sub_koeff[icentr])
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.SetMarkerSize(1)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")